# Quality of life, MCI metrics

## Measure change of life behaviours

* Activities of Daily Living
* Mobility
* Sociability
* *Cognition* (not measured)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reasoning.presets.dataset import load_default_datasets

In [3]:
import pandas as pd
import numpy as np

from bokeh.plotting import output_notebook

from reasoning.data_analysis.data_viz import plot
from reasoning.data_analysis.update_csv import full_csv_update
from reasoning.data_analysis.data_toolbox import load_activities, load_events, load_groundtruth
from reasoning.presets.dataset import load_default_datasets

from datetime import timedelta

output_notebook()

Loading BokehJS ...

In [6]:
# events, activities, groundtruth = load_default_datasets(update_csv_files=True)

path_to_dataset_E = '../new_dataset/Dataset-Extend'
path_to_dataset_N = '../new_dataset/Dataset-Normandie'

# events_N, activities_N = load_default_datasets(update_csv_files=False,
#                    path=path_to_dataset_N,
#                    event_dataset='public.event.csv',
#                    activity_dataset='public.activity.csv',
#                    delim=';',
#                    patients='N')

In [7]:
# events_E, activities_E = load_default_datasets(update_csv_files=False,
#                     path=path_to_dataset_E,
#                     event_dataset='public.event.csv',
#                     activity_dataset='public.activity.csv',
#                     delim=';',
#                     patients='E')

In [115]:
import pickle

To export:

In [12]:
events_E, activities_E, events_N, activities_N = (1,2,3,4)
object_to_export = (events_E, activities_E, events_N, activities_N)
with open('filename', 'wb') as f:
    pickle.dump(object_to_export, f)

To load:

In [14]:
with open('filename', 'rb') as f:
    events_E, activities_E, events_N, activities_N = pd.read_pickle(f)
    #events_E, activities_E, events_N, activities_N = pickle.load(f)
    
print(events_E, activities_E, events_N, activities_N)

(                                 id               createdAt  \
patient date                                                  
B       2017-01-11 14:26:40  315611  2017-01-11 14:26:41+00   
        2017-01-11 14:26:54  315615  2017-01-11 14:26:55+00   
        2017-01-11 14:26:59  315616  2017-01-11 14:26:59+00   
        2017-01-11 14:27:01  315618  2017-01-11 14:27:01+00   
        2017-01-11 14:27:38  315622  2017-01-11 14:27:38+00   
        2017-01-11 14:27:47  315624  2017-01-11 14:27:47+00   
        2017-01-11 14:27:58  315626  2017-01-11 14:27:58+00   
        2017-01-11 14:28:12  315629  2017-01-11 14:28:13+00   
        2017-01-11 14:28:23  315631  2017-01-11 14:28:24+00   
        2017-01-11 14:28:42  315634  2017-01-11 14:28:42+00   
        2017-01-11 14:29:09  315643  2017-01-11 14:29:09+00   
        2017-01-11 14:29:24  315647  2017-01-11 14:29:24+00   
        2017-01-11 14:29:35  315649  2017-01-11 14:29:35+00   
        2017-01-11 14:29:46  315650  2017-01-11 14:29:

In [25]:
events = pd.concat([events_E, events_N])
activities = pd.concat([activities_E, activities_N])

print("Events: ", events_E.shape, events_N.shape, "->", events.shape)

print("Activities: ", activities_E.shape, activities_N.shape, "->", activities.shape)

('Events: ', (86242, 5), (427651, 5), '->', (513893, 5))
('Activities: ', (3318, 8), (746295, 8), '->', (749613, 8))


In [98]:
events.index.names

FrozenList([u'patient', u'date'])

In [99]:
events.head()

id               createdAt  \
patient date                                                  
B       2017-01-11 14:26:40  315611  2017-01-11 14:26:41+00   
        2017-01-11 14:26:54  315615  2017-01-11 14:26:55+00   
        2017-01-11 14:26:59  315616  2017-01-11 14:26:59+00   
        2017-01-11 14:27:01  315618  2017-01-11 14:27:01+00   
        2017-01-11 14:27:38  315622  2017-01-11 14:27:38+00   

                                          updatedAt           sensor value  
patient date                                                                
B       2017-01-11 14:26:40  2017-01-11 14:26:41+00  Bathroom Motion   Off  
        2017-01-11 14:26:54  2017-01-11 14:26:55+00  Bathroom Motion    On  
        2017-01-11 14:26:59  2017-01-11 14:26:59+00  Bathroom Motion    On  
        2017-01-11 14:27:01  2017-01-11 14:27:01+00  Bathroom Motion    On  
        2017-01-11 14:27:38  2017-01-11 14:27:38+00  Bathroom Motion    On

## Events/day

In [106]:
def get_count_daily_events_user(events, user):
    return events.loc[user].resample('24h', base=10)['id'].count()
#events_user = events.loc[user].resample('1D')['id'].count()
# or:
# events_user = events.loc[user].groupby(pd.Grouper(freq='1D'))['id'].count()

In [113]:
user = 'A'
events_user = get_count_daily_events_user(events, user)
events_user.head()

date
2016-03-29 10:00:00    271
2016-03-30 10:00:00    263
2016-03-31 10:00:00    509
2016-04-01 10:00:00    358
2016-04-02 10:00:00    414
Freq: 24H, Name: id, dtype: int64

In [114]:
from bokeh.plotting import figure, show

p = figure(title=('Number of events per day user %s' % user), x_axis_label='Day', y_axis_label='Events', x_axis_type="datetime",
          tools='box_select,xpan,xwheel_zoom,reset,save', height=400)

p.line(events_user.index, events_user.values)
#p.segment(x0=fall_days, y0=5,
#          x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend='Falls', color='red')

show(p)

## Energy rate

In [19]:
events.reset_index(level=0, drop=True)

,id,createdAt,updatedAt,sensor,value
date,,,,,
2017-01-11 14:26:40,315611,2017-01-11 14:26:41+00,2017-01-11 14:26:41+00,Bathroom Motion,Off
2017-01-11 14:26:54,315615,2017-01-11 14:26:55+00,2017-01-11 14:26:55+00,Bathroom Motion,On
2017-01-11 14:26:59,315616,2017-01-11 14:26:59+00,2017-01-11 14:26:59+00,Bathroom Motion,On
2017-01-11 14:27:01,315618,2017-01-11 14:27:01+00,2017-01-11 14:27:01+00,Bathroom Motion,On
2017-01-11 14:27:38,315622,2017-01-11 14:27:38+00,2017-01-11 14:27:38+00,Bathroom Motion,On
2017-01-11 14:27:47,315624,2017-01-11 14:27:47+00,2017-01-11 14:27:47+00,Bathroom Motion,On
2017-01-11 14:27:58,315626,2017-01-11 14:27:58+00,2017-01-11 14:27:58+00,Bathroom Motion,On
2017-01-11 14:28:12,315629,2017-01-11 14:28:13+00,2017-01-11 14:28:13+00,Bathroom Motion,On
2017-01-11 14:28:23,315631,2017-01-11 14:28:24+00,2017-01-11 14:28:24+00,Bathroom Motion,On


In [21]:
events.loc['C']

,id,createdAt,updatedAt,sensor,value
date,,,,,
2016-03-29 16:28:22,366075,2016-03-29 18:28:24+02,2016-03-29 18:28:24+02,Bedroom Motion,On
2016-03-29 16:29:24,366078,2016-03-29 18:29:24+02,2016-03-29 18:29:24+02,Bedroom Motion,Off
2016-03-29 16:30:44,366080,2016-03-29 18:30:45+02,2016-03-29 18:30:45+02,Bedroom Motion,On
2016-03-29 16:31:46,366081,2016-03-29 18:31:47+02,2016-03-29 18:31:47+02,Bedroom Motion,Off
2016-03-29 16:40:40,366083,2016-03-29 18:40:40+02,2016-03-29 18:40:40+02,Bedroom Motion,On
2016-03-29 16:41:23,366084,2016-03-29 18:41:23+02,2016-03-29 18:41:23+02,Bedroom Motion,On
2016-03-29 16:41:35,366085,2016-03-29 18:41:35+02,2016-03-29 18:41:35+02,B4EB00,Normal
2016-03-29 16:41:53,366086,2016-03-29 18:41:55+02,2016-03-29 18:41:55+02,Bedroom Motion,On
2016-03-29 16:42:29,366087,2016-03-29 18:42:29+02,2016-03-29 18:42:29+02,Bedroom Motion,On


In [22]:
# TODO: Make it work as groupby

def aggregate_events(events):
    aggregated_events = events.reset_index(level=0, drop=True).value.resample('D', how='count')
    aggregated_events = aggregated_events[aggregated_events != 0]
    min_events, max_events = min(aggregated_events), max(aggregated_events)
    aggregated_events = aggregated_events * 100 / max_events
    return aggregated_events

events.groupby(level=0).apply(aggregate_events)

/home/pau/.virtualenvs/butchery/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  after removing the cwd from sys.path.


patient  date      
A        2016-03-29     4.050523
         2016-03-30    15.113240
         2016-03-31    15.810105
         2016-04-01    20.993031
         2016-04-02    15.113240
         2016-04-03    18.292683
         2016-04-04    16.114983
         2016-04-05    22.168990
         2016-04-06    19.860627
         2016-04-07    23.214286
         2016-04-08    19.076655
         2016-04-09    17.247387
         2016-04-10    16.289199
         2016-04-11    22.648084
         2016-04-12    28.048780
         2016-04-13    17.378049
         2016-04-14    25.435540
         2016-04-15    22.256098
         2016-04-16    22.517422
         2016-04-17    18.858885
         2016-04-18    22.909408
         2016-04-19    24.520906
         2016-04-20    11.977352
         2016-04-21    18.336237
         2016-04-22    26.611498
         2016-04-23    14.808362
         2016-04-24    19.294425
         2016-04-25    22.691638
         2016-04-26    25.914634
         2016-04-27    

In [23]:
aggregated_events = events.loc['C'].value.resample('D', how='count')
min_events, max_events = min(aggregated_events), max(aggregated_events)
aggregated_events = aggregated_events * 100 / max_events
aggregated_events

/home/pau/.virtualenvs/butchery/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  """Entry point for launching an IPython kernel.


date
2016-03-29     5.274725
2016-03-30    20.879121
2016-03-31    22.032967
2016-04-01    23.571429
2016-04-02    25.989011
2016-04-03    21.538462
2016-04-04    21.483516
2016-04-05    21.208791
2016-04-06    17.582418
2016-04-07    19.175824
2016-04-08    22.747253
2016-04-09    22.967033
2016-04-10    18.406593
2016-04-11    17.857143
2016-04-12    20.000000
2016-04-13    16.758242
2016-04-14    22.802198
2016-04-15    28.351648
2016-04-16    21.098901
2016-04-17    20.109890
2016-04-18    20.164835
2016-04-19    20.054945
2016-04-20    22.747253
2016-04-21    22.142857
2016-04-22    26.538462
2016-04-23    22.582418
2016-04-24    17.307692
2016-04-25    20.000000
2016-04-26    21.703297
2016-04-27    26.758242
                ...    
2016-12-13    16.098901
2016-12-14    17.087912
2016-12-15    15.274725
2016-12-16    21.538462
2016-12-17    20.494505
2016-12-18    15.934066
2016-12-19    16.648352
2016-12-20    19.175824
2016-12-21    22.527473
2016-12-22    18.241758
2016-12-23 

In [26]:
def custom(my_day):
    a = my_day.copy()
    a.loc[:, 'duration'] = a.index - a.since
    falls = a[(a.activity == "Fall") & (a.duration >= timedelta(minutes=5))]
    return not falls.empty

a = activities.loc['C']
fall_per_day = a.resample('D', how=custom).activity
fall_days = fall_per_day[fall_per_day].index

/home/pau/.virtualenvs/butchery/lib/python2.7/site-packages/ipykernel_launcher.py:8: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  


In [36]:
from bokeh.plotting import figure, show

p = figure(title="Amount of activity per day", x_axis_label='Day', y_axis_label='Events', x_axis_type="datetime",
          tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.line(aggregated_events.index, aggregated_events.values)
p.segment(x0=fall_days, y0=5,
          x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Events/dia

In [47]:
patient_index = events.index.get_level_values('patient').unique()
print("Patients: ", sorted(list(patient_index)))

('Patients: ', ['A', 'B', 'C', 'D', 'E'])


In [63]:
#g = events.resample('D', level=1)
#g.count()
#events.groupby(pd.Grouper(freq='1D', level=1)).count()
events.resample('3T', level=1).count()

,id,createdAt,updatedAt,sensor,value
date,,,,,
2016-03-29 16:27:00,5,5,5,5,5
2016-03-29 16:30:00,4,4,4,4,4
2016-03-29 16:33:00,0,0,0,0,0
2016-03-29 16:36:00,0,0,0,0,0
2016-03-29 16:39:00,4,4,4,4,4
2016-03-29 16:42:00,3,3,3,3,3
2016-03-29 16:45:00,25,25,25,25,25
2016-03-29 16:48:00,23,23,23,23,23
2016-03-29 16:51:00,6,6,6,6,6


## Time outdoor / Time indoor

In [ ]:
activities.loc['I'].activity.unique()

In [ ]:
def outside_time_estimator(my_day):
    outside = my_day[my_day.activity == "Go Out"]
    time_spend_outside = outside.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_outside.empty:
        return timedelta(seconds=0)
    return time_spend_outside.sum()

a = activities.loc['J']
time_spend_outside = a.resample("D", how=outside_time_estimator).activity
time_spend_outside

max_bound = time_spend_outside[time_spend_outside != 0].quantile(.9)
time_spend_outside.loc[time_spend_outside > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Time spent outside per day", x_axis_label='Day', y_axis_label='Outside time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.line(time_spend_outside.index, (time_spend_outside / np.timedelta64(1, 's')).values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Time spent in bathroom

In [64]:
from dateutil.parser import parse

def toilet_duration_estimator(my_day):
    outside = my_day[my_day.activity == "Go Toilet"]
    time_spend_outside = outside.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_outside.empty:
        return timedelta(seconds=0)
    return time_spend_outside.sum()

def toilet_count_estimator(my_day):
    outside = my_day[my_day.activity == "Go Toilet"]
    return outside.count()

a = activities.loc['J']
duration_spend_in_toilet = a.resample("D", how=toilet_duration_estimator).activity
duration_spend_in_toilet

max_bound = duration_spend_in_toilet[duration_spend_in_toilet != 0].quantile(.9)
duration_spend_in_toilet.loc[duration_spend_in_toilet > max_bound] = max_bound

toilet_occurences = a.resample("D", how=toilet_count_estimator).activity
toilet_occurences

max_bound = toilet_occurences[toilet_occurences != 0].quantile(.9)
toilet_occurences.loc[toilet_occurences > max_bound] = max_bound

KeyError: 'the label [J] is not in the [index]'

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

from bokeh.models.formatters import NumeralTickFormatter

p = figure(title="Time spent in toilet per day", x_axis_label='Day', y_axis_label='Toilet time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=toilet_occurences.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_toilet.index, (duration_spend_in_toilet / np.timedelta64(1, 's')).values)

p.line(toilet_occurences.index, toilet_occurences.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Toilet by night

In [ ]:
hours = a.index.get_level_values('date').map(lambda x: x.hour)
hours
a2 = a[(hours >= 20) | (hours < 10)].copy()
a2.loc[:, 'shifted_hours'] = a2.index.get_level_values('date') + timedelta(hours=4)
a2.loc[:, 'since'] = a2.since + timedelta(hours=4)
a2.reset_index(drop=True, inplace=True)
a2.rename(columns={'shifted_hours': 'date'}, inplace=True)
a2.set_index('date', inplace=True)

max_bound = duration_spend_in_toilet[duration_spend_in_toilet != 0].quantile(.9)
duration_spend_in_toilet.loc[duration_spend_in_toilet > max_bound] = max_bound

duration_spend_in_toilet_by_night = a2.resample("D", how=toilet_duration_estimator).activity
duration_spend_in_toilet_by_night

max_bound = duration_spend_in_toilet_by_night[duration_spend_in_toilet_by_night != 0].quantile(.9)
duration_spend_in_toilet_by_night.loc[duration_spend_in_toilet_by_night > max_bound] = max_bound

toilet_occurences_by_night = a2.resample("D", how=toilet_count_estimator).activity
toilet_occurences_by_night

max_bound = toilet_occurences_by_night[toilet_occurences_by_night != 0].quantile(.9)
toilet_occurences_by_night.loc[toilet_occurences_by_night > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

p = figure(title="Time spent in toilet per night", x_axis_label='Day', y_axis_label='Toilet time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=toilet_occurences_by_night.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_toilet_by_night.index, (duration_spend_in_toilet_by_night / np.timedelta64(1, 's')).values)

p.line(toilet_occurences_by_night.index, toilet_occurences_by_night.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Kitchen

In [ ]:
def kitchen_duration_estimator(my_day):
    in_kitchen = my_day[my_day.activity.isin(['Eat Meal', 'Cook Meal'])]
    time_spend_in_kitchen = in_kitchen.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_in_kitchen.empty:
        return timedelta(seconds=0)
    return time_spend_in_kitchen.sum()

def kitchen_count_estimator(my_day):
    in_kitchen = my_day[my_day.activity.isin(['Eat Meal', 'Cook Meal'])]
    return in_kitchen.count()

a = activities.loc['J']
duration_spend_in_kitchen = a.resample("D", how=kitchen_duration_estimator).activity
duration_spend_in_kitchen

max_bound = duration_spend_in_kitchen[duration_spend_in_kitchen != 0].quantile(.9)
duration_spend_in_kitchen.loc[duration_spend_in_kitchen > max_bound] = max_bound

kitchen_occurences = a.resample("D", how=kitchen_count_estimator).activity
kitchen_occurences

max_bound = kitchen_occurences[kitchen_occurences != 0].quantile(.9)
kitchen_occurences.loc[kitchen_occurences > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

p = figure(title="Time spent in kitchen per day", x_axis_label='Day', y_axis_label='Kitchen time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=kitchen_occurences.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_kitchen.index, (duration_spend_in_kitchen / np.timedelta64(1, 's')).values)

p.line(kitchen_occurences.index, kitchen_occurences.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Sleeping

In [ ]:
def sleeping_duration_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Sleep']
    time_spend_sleeping = sleeping.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_sleeping.empty:
        return timedelta(seconds=0)
    return time_spend_sleeping.sum()

def sleeping_count_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Sleep']
    return sleeping.count()

def nap_duration_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Nap']
    time_spend_sleeping = sleeping.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_sleeping.empty:
        return timedelta(seconds=0)
    return time_spend_sleeping.sum()

def nap_count_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Nap']
    return sleeping.count()

a = activities.loc['J']
duration_spend_sleeping = a.resample("D", how=sleeping_duration_estimator).activity
duration_spend_sleeping

sleeping_occurences = a.resample("D", how=sleeping_count_estimator).activity
sleeping_occurences

a = activities.loc['J']
duration_spend_nap = a.resample("D", how=nap_duration_estimator).activity
duration_spend_nap

nap_occurences = a.resample("D", how=nap_count_estimator).activity
nap_occurences

In [ ]:
sleep_df = pd.DataFrame(data={'nap': duration_spend_nap.values, 'sleep': duration_spend_sleeping.values},
                        index=duration_spend_nap.index)

In [ ]:
def stacked(df, categories):
    areas = dict()
    last = np.zeros(len(df[categories[0]]))
    for cat in categories:
#         print(last, df[cat])
        next = last + df[cat] / np.timedelta64(1, 's')
        areas[cat] = np.hstack((last[::-1], next))
        last = next
    return areas

In [ ]:
brewer['Spectral'][8]

In [ ]:
from bokeh.palettes import brewer

p = figure(title="Time spent sleeping per day", x_axis_label='Day', y_axis_label='Sleeping time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

# p.extra_y_ranges = {"occurences": Range1d(start=0, end=sleeping_occurences.max())}
# p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

x2 = np.hstack((sleep_df.index[::-1], sleep_df.index))

categories = ['nap', 'sleep']
areas = stacked(sleep_df, categories)

colors = ['#fee08b', '#fdae61']

p.patches([x2] * len(areas), [areas[cat] for cat in categories],
          color=colors, alpha=0.8, line_color=None)

# p.line(duration_spend_sleeping.index, (duration_spend_sleeping / np.timedelta64(1, 's')).values)

# p.line(sleeping_occurences.index, sleeping_occurences.values, color="Red", y_range_name="occurences")

show(p)

# Active

In [ ]:
def active_duration_estimator(my_day):
    active = my_day[~my_day.activity.isin(['Go Out', 'Sleep', 'Nap', 'Inactive'])]
    time_spend_active = active.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_active.empty:
        return timedelta(seconds=0)
    return time_spend_active.sum()

a = activities.loc['J']
duration_active = a.resample("D", how=active_duration_estimator).activity
duration_active

max_bound = duration_active[duration_active != 0].quantile(.9)
duration_active.loc[duration_active > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Time spent being active per day", x_axis_label='Day', y_axis_label='Active time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.line(duration_active.index, (duration_active / np.timedelta64(1, 's')).values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Changing rooms

In [ ]:
def change_rooms_estimator(my_day):
    e = my_day[my_day.value == "On"].copy()
    e.loc[:, 'previous_location'] = e.sensor.shift()
    e = e[(e.sensor != e.previous_location)]
    return e.count()

e = events.loc['J']
change_rooms = e.resample("D", how=change_rooms_estimator).sensor
change_rooms

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Numer of times changing rooms per day", x_axis_label='Day', y_axis_label='Changing rooms',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.line(change_rooms.index, change_rooms.values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Misc.

In [ ]:
plot(activities=activities, patients=['J'])

In [ ]:
events.loc['J']

In [ ]:
plot(events=events.head(1000))